# 문서 분할

In [ ]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

문서를 아무렇게나 분할하게 되면 그 의미가 소실될 가능성이 굉장히 높습니다.  
보통은 길이를 기준으로 분할하게 되는데 의미있는 단위가 뭉개지는 것이죠.  
예를 들어 '나는 어제 저녁에 치킨을 먹었다.' 라는 문장이 '나는 어제 저녁에 치'와 '킨을 먹었다.'로 쪼개지게 되면 '치킨'이라는 단어 정보가 사라지게 되는 것입니다.

따라서 일반적으로 문서를 분할할 때는 중복 구간을 설정하여 원래 의미가 사라질 가능성을 최소화 합니다.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

In [ ]:
chunk_size =26
chunk_overlap = 4

In [ ]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)
c_splitter = CharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)

Why doesn't this split the string below?

In [ ]:
text1 = 'abcdefghijklmnopqrstuvwxyz'

In [ ]:
r_splitter.split_text(text1)

In [ ]:
text2 = 'abcdefghijklmnopqrstuvwxyzabcdefg'

특정 길이를 넘어가면 설정된 중복만큼 앞뒤 글자가 붙어 있는 것을 확인할 수 있습니다

In [ ]:
r_splitter.split_text(text2)

공백 또한 문자로 취급되기 때문에 중복 문자수에 포함됩니다.

In [ ]:
text3 = "a b c d e f g h i j k l m n o p q r s t u v w x y z"

In [ ]:
r_splitter.split_text(text3)

c_splitter의 기본 구분자(sperator)는 '\n'이므로 변경하지 않으면 다음과 같이 분할되지 않습니다.

In [ ]:
c_splitter.split_text(text3)

In [ ]:
c_splitter = CharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    separator = ' '
)
c_splitter.split_text(text3)

자신만의 예시를 만들어 보세요!

## 재귀적 분할 세부사항

일반적인 텍스트에 대해서는 `RecursiveCharacterTextSplitter`의 사용이 권장됩니다.

In [ ]:
some_text = """When writing documents, writers will use document structure to group content. \
This can convey to the reader, which idea's are related. For example, closely related ideas \
are in sentances. Similar ideas are in paragraphs. Paragraphs form a document. \n\n  \
Paragraphs are often delimited with a carriage return or two carriage returns. \
Carriage returns are the "backslash n" you see embedded in this string. \
Sentences have a period at the end, but also, have a space.\
and words are separated by space."""

In [ ]:
len(some_text)

주어진 `chunk_size` 이내의 단위로 쪼갤 때, 만약 처음의 구분자로도 부족하다면  
리스트 내 다음 구분자를 기준으로 분할하게 됩니다.

아래 예시에서는 '두 줄 바꿈' -> '줄 바꿈' -> '띄어 쓰기' -> '글자' 단위로 분할이 적용됩니다.

In [ ]:
c_splitter = CharacterTextSplitter(
    chunk_size=450,
    chunk_overlap=0,
    separator = ' '
)
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=450,
    chunk_overlap=0, 
    separators=["\n\n", "\n", " ", ""]
)

In [ ]:
c_splitter.split_text(some_text)

In [ ]:
r_splitter.split_text(some_text)

이번에는 chunk size를 조금 줄이고 구분 단위를 늘려보도록 하죠.

In [ ]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=0,
    separators=["\n\n", "\n", "\. ", " ", ""]
)
r_splitter.split_text(some_text)

In [ ]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=0,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""]
)
r_splitter.split_text(some_text)

앞선 강의에서 다뤘던 것처럼 PDF 파일을 불러와서 이를 분할할수도 있습니다.

In [ ]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("docs/cs229_lectures/MachineLearning-Lecture01.pdf")
pages = loader.load()

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=150,
    length_function=len
)

In [ ]:
docs = text_splitter.split_documents(pages)

In [ ]:
len(docs)

In [ ]:
len(pages)

마찬가지로 노션의 데이터를 불러와 분할하는 예시입니다.

In [ ]:
from langchain.document_loaders import NotionDirectoryLoader
loader = NotionDirectoryLoader("docs/Notion_DB")
notion_db = loader.load()

In [ ]:
docs = text_splitter.split_documents(notion_db)

In [ ]:
len(notion_db)

In [ ]:
len(docs)

## 토큰 분할

만약 원한다면 토큰 단위를 기준으로 분할하는 것도 가능합니다.

이는 LLM이 토큰을 기준으로 context window를 갖기 때문에 굉장히 유용합니다.

토큰은 보통 ~4글자입니다.

In [ ]:
from langchain.text_splitter import TokenTextSplitter

In [ ]:
text_splitter = TokenTextSplitter(chunk_size=1, chunk_overlap=0)

In [ ]:
text1 = "foo bar bazzyfoo"

토큰 단위로 분할되는 것을 확인할 수 있습니다.

In [ ]:
text_splitter.split_text(text1)

In [ ]:
text_splitter = TokenTextSplitter(chunk_size=10, chunk_overlap=0)

In [ ]:
docs = text_splitter.split_documents(pages)

메타데이터가 분할 단위로 포함되어 있음 또한 확인할 수 있습니다.

In [ ]:
docs[0]

In [ ]:
pages[0].metadata

## 문맥 인지 분할

덩어리(Chunking)은 텍스트에 대해 일반적 문맥을 보존을 목표로 수행됩니다.

텍스트 분할은 주로 문장이나 다른 구획문자 단위로 이뤄집니다.
하지만 많은 문서들이(마크다운과 같은) 헤더와 같이 구분을 위해 명시적으로 사용되는 구조를 갖추고 있습니다.


우리는 우리의 덩어리(chunk)에서 헤더 메타데이터를 보존하기 위해 `MarkdownHeaderTextSplitter`를 사용합니다.

In [ ]:
from langchain.document_loaders import NotionDirectoryLoader
from langchain.text_splitter import MarkdownHeaderTextSplitter

In [ ]:
markdown_document = """# Title\n\n \
## Chapter 1\n\n \
Hi this is Jim\n\n Hi this is Joe\n\n \
### Section \n\n \
Hi this is Lance \n\n 
## Chapter 2\n\n \
Hi this is Molly"""

In [ ]:
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

In [ ]:
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on
)
md_header_splits = markdown_splitter.split_text(markdown_document)

In [ ]:
md_header_splits[0]

In [ ]:
md_header_splits[1]

노션 데이터베이스와 같은 실제 마크다운 파일로 시도해보세요.

In [ ]:
loader = NotionDirectoryLoader("docs/Notion_DB")
docs = loader.load()
txt = ' '.join([d.page_content for d in docs])

In [ ]:
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
]
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on
)

In [ ]:
md_header_splits = markdown_splitter.split_text(txt)

In [ ]:
md_header_splits[0]